<a href="https://colab.research.google.com/github/kwanglo/mge51101-20195171/blob/master/final_project/multi_sentiment_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!pip3 install konlpy
!pip3 install soynlp

In [0]:
import os
import re

from sklearn import datasets, model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Twitter

import pandas as pd
import numpy as np

# Data preprocessing

In [0]:
path='/gdrive/My Drive/Colab Notebooks/Final Project/dataset/'
df = pd.read_excel(path+"한국어_단발성_대화_데이터셋.xlsx")

In [0]:
df = df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4','공포',5468])

In [0]:
# Char to int
df.loc[(df.Emotion == "중립"),"Emotion"] = 0
df.loc[(df.Emotion == "공포"),"Emotion"] = 1
df.loc[(df.Emotion == "놀람"),"Emotion"] = 2
df.loc[(df.Emotion == "분노"),"Emotion"] = 3
df.loc[(df.Emotion == "슬픔"),"Emotion"] = 4
df.loc[(df.Emotion == "행복"),"Emotion"] = 5
df.loc[(df.Emotion == "혐오"),"Emotion"] = 6

In [0]:
data = df

In [0]:
# 숫자로 이미지 나눌때 
#중립
Neutral = data[data["Emotion"] == 0]
#공포
Fear = data[data["Emotion"] == 1]
Fear.head()
#놀람
Surprise = data[data["Emotion"] == 2]
#분노
Anger = data[data["Emotion"] == 3]
#슬픔
Sad = data[data["Emotion"] == 4]
#행복
Happy = data[data["Emotion"] == 5]
#혐오
Disgust = data[data["Emotion"] == 6]

In [0]:
rnd_num = 2020
Fear_train = Fear.sample(frac=0.7, random_state=rnd_num)
Fear_test = Fear.drop(Fear_train.index)

Surprise_train = Surprise.sample(frac=0.7, random_state=rnd_num)
Surprise_test = Surprise.drop(Surprise_train.index)

Anger_train = Anger.sample(frac=0.7, random_state=rnd_num)
Anger_test = Anger.drop(Anger_train.index)

Sad_train = Sad.sample(frac=0.7, random_state=rnd_num)
Sad_test = Sad.drop(Sad_train.index)

Neutral_train = Neutral.sample(frac=0.7, random_state=rnd_num)
Neutral_test = Neutral.drop(Neutral_train.index)

Happy_train = Happy.sample(frac=0.7, random_state=rnd_num)
Happy_test = Happy.drop(Happy_train.index)

Disgust_train = Disgust.sample(frac=0.7, random_state=rnd_num)
Disgust_test = Disgust.drop(Disgust_train.index)

train = pd.concat([Fear_train,Surprise_train,Anger_train,Sad_train,Neutral_train,Happy_train,Disgust_train])
test = pd.concat([Fear_test,Surprise_test,Anger_test,Sad_test,Neutral_test,Happy_test,Disgust_test])

In [0]:
from soynlp.tokenizer import MaxScoreTokenizer
from soynlp.normalizer import *
import re
from konlpy.tag import Okt

def tokenizer(text): # create a tokenizer function
    okt = Okt()
    text = only_hangle(text)
    text = repeat_normalize(text, num_repeats = 2)
    x = okt.morphs(text , stem= True)
    return x

In [0]:
stop_words_set = pd.read_csv(path+'stopwords100.txt',header = 0, delimiter = '\t', quoting = 3)
stop_words= (list(stop_words_set['aa']))
stop_words2 = ['은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', '주', '등', '한']
stop_words.extend(stop_words)

In [0]:
import torch
from torchtext import data
from torchtext import datasets
from soynlp.tokenizer import MaxScoreTokenizer
SEED = 3432

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


TEXT = data.Field(tokenize=tokenizer, stop_words = stop_words)
LABEL = data.LabelField()

In [0]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split

# Naive Bayes

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB # 다항분포 나이브 베이즈 모델
from sklearn.metrics import accuracy_score #정확도 계산

In [0]:
dtmvector = CountVectorizer()
X_train_dtm = dtmvector.fit_transform(train_set['Sentence'])
print(X_train_dtm.shape)

(27016, 70785)


In [0]:
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(X_train_dtm)
print(X_train_dtm.shape)

(27016, 70785)


In [0]:
X_train = train_set['Sentence']
X_test = test_set['Sentence']
Y_train = train_set['Emotion']
Y_test = test_set['Emotion']
label_total = df['Emotion']

In [0]:
train_sentence = Y_train.to_list()
test_sentence = Y_test.to_list()

In [0]:
len(train_sentence)

27016

In [0]:
mod = MultinomialNB()
mod.fit(tfidfv, train_sentence)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
X_test_dtm = dtmvector.transform(test_set['Sentence']) #테스트 데이터를 DTM으로 변환
tfidfv_test = tfidf_transformer.transform(X_test_dtm) #DTM을 TF-IDF 행렬로 변환

predicted = mod.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(test_sentence, predicted)) #예측값과 실제값 비교

정확도: 0.37122128174123337


In [0]:
#추가적으로 감정별로 정확도 차이를 비교해봐도 좋을듯?